# 14. 반복형, 반복자, 제너레이터
## 14.1 Sentence  take #1: 단어 시퀀스

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 14.1.1 Sequence가 반복 가능한 이유: `iter()` 함수
---
파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출한다.
`iter()` 내장 함수는 다음 과정을 수행한다.
1. 객체가 `__iter__()` 메서드를 구현하는지 확인하고, 이 메서드를 호출해서 반복자를 가져온다.
2. `__iter__()` 메서드가 구현되어 있지 않지만 `__getitem__()`이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자를 생성한다.
3. 이 과정이 모두 실패하면 파이썬은 `'TypeError: 'C' object is not iterable'`이라는 메시지와 함께 TypeError가 발생한다. 여기서 C는 대상 객체의 클래스다.

In [5]:
class Foo:
    def __iter__(self):
        pass

from collections import abc
issubclass(Foo, abc.Iterable)

True

In [6]:
f = Foo()
isinstance(f, abc.Iterable)

True

## 14.2 반복형과 반복자

| 반복형 |
| ---------- |
|`iter()` 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 `__iter__()` 메서드를 구현하는 객체는 반복형이다. 0에서 시작하는 인덱스를 받는 `__getitem__()` 메서드를 구현하는 객체인 시퀀스도 마찬가지다.|


In [7]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [9]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


**`__next__()`**

다음에 사용할 항목을 반환한다. 더 이상 항목이 남아 있지 않으면 StopIteration을 발생시킨다.



**`__iter__()`**

self를 반환한다. 그러면 for 루프 등 반복형이 필요한 곳에 반복자를 상요할 수 있게 해준다.

In [10]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it

In [11]:
next(it)

'Pig'

In [12]:
next(it)

'and'

In [13]:
next(it)

'Pepper'

In [14]:
next(it)

StopIteration: 

In [15]:
list(it)

[]

In [16]:
list(iter(s3))

['Pig', 'and', 'Pepper']

## 14.3 Sentence take #2: 고전적인 반복자

In [23]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [24]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [25]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [26]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 14.3.1 Sentence를 반복자로 만들기: 좋지 않은 생각
---
반복형과 반복자를 만드는 데 있어서 흔히 발생하는 오류는 둘을 혼동하기 때문에 발생한다. 간단히 정리하면, 반복형은 호출될 때마다 반복자를 새로 생성하는 `__iter__()` 메서드를 가지고 있다. 반복자는 개별 항목을 반환하는 `__next__()` 메서드와 `self`를 반환하는 `__iter__()` 메서드를 가지고 있다.

**따라서 반복자는 반복형이지만, 반복형은 반복자가 아니다.**

## 14.4 Sentence take #3: 제너레이터 함수

In [27]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

In [28]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [29]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [30]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [31]:
def gen_123():
    yield 1
    yield 2
    yield 3

gen_123

<function __main__.gen_123>

In [32]:
gen_123()

<generator object gen_123 at 0x104599258>

In [33]:
for i in gen_123():
    print(i)

1
2
3


In [34]:
g = gen_123()
next(g)

1

In [35]:
next(g)

2

In [36]:
next(g)

3

In [37]:
next(g)

StopIteration: 

In [38]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


## 14.5 Sentence take #4: 느긋한 구현

In [40]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

In [41]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)
list(s)

The
time
has
come
the
Walrus
said


['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.6 Sentence take #5: 제너레이터 표현식

In [42]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [43]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [44]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x1045998e0>

In [45]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [46]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [47]:
s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)
list(s)

The
time
has
come
the
Walrus
said


['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.10 파이썬 3.3의 새로운 구문: `yield from`

In [48]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i
            
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [51]:
def chain(*iterables):
    for i in iterables:
        yield from i

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]